In [1]:
import numpy as np
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

EC kcat

In [ ]:
Brenda_sabio_combined_data = json_load('./data/Brenda_sabio_combined.json' )
ec_max_df = pd.DataFrame()
for eachec in Brenda_sabio_combined_data.keys():
    kcat_value_list = []
    for eachkey2 in Brenda_sabio_combined_data[eachec].keys():
        if eachkey2 == 'SOURCE':
            ec_max_df.loc[eachec,'SOURCE'] = Brenda_sabio_combined_data[eachec][eachkey2]  
        elif  isinstance(Brenda_sabio_combined_data[eachec][eachkey2],dict):
            for eachkey3 in Brenda_sabio_combined_data[eachec][eachkey2].keys():
                #print(eachec,eachkey2,eachkey3,Brenda_sabio_combined_data[eachec][eachkey2][eachkey3])
                if  isinstance(Brenda_sabio_combined_data[eachec][eachkey2][eachkey3],dict):
                    kcat_value_list.append(np.max([i for i in Brenda_sabio_combined_data[eachec][eachkey2][eachkey3].values()]))
                else:
                    kcat_value_list.append(np.max(Brenda_sabio_combined_data[eachec][eachkey2][eachkey3]))
    #print(np.max(kcat_value_list))
    ec_max_df.loc[eachec,'kcat'] = np.max(kcat_value_list)
ec_max_df.to_csv('./analysis/ec_max_kcat_df.csv')

gene kcat

In [8]:
import numpy as np
gene_id_data_mapping_data = json_load('./data/gene_id_data_mapping.json' )
gene_max_df = pd.DataFrame()
for eachgene in gene_id_data_mapping_data.keys():
    gene_max_df.loc[eachgene,'kcat'] = np.max(gene_id_data_mapping_data[eachgene]['kcats'])

gene_max_df.to_csv('./analysis/gene_max_kcat_df.csv')

go kcat

In [2]:
import re
json_model_path = "./model/iML1515_irr_enz_constraint_kapp.json"
enz_model=get_enzyme_constraint_model(json_model_path)
reaction_max_df = pd.DataFrame()
GO_ec_kcat = pd.read_csv('./analysis/GO_ec_kcat.csv')
ec_max_kcat_df = pd.read_csv('./analysis/ec_max_kcat_df.csv', index_col=0)
gene_max_kcat_df = pd.read_csv('./analysis/ec_max_kcat_df.csv', index_col=0)
gene_GO = pd.read_csv('./data/gene_go.csv')
GO_level_space_out = pd.read_csv('./analysis/GO_level_space_out.csv', index_col=0)

for reaction in enz_model.reactions:
    if "ec-code" not in reaction.annotation.keys():
        if re.search('EX_',reaction):
            pass
        else:
            # reaction gpr 
            gpr_list = reaction.gene_reaction_rule.split(' ')
            if len(gpr_list)>0:
                gene_list=[]
                gene_kcat_list=[]
                for eachdata in gpr_list:
                    if len(eachdata)>4:
                        if eachdata not in gene_list:
                            #使用 gene-kcat
                            if eachdata in gene_max_kcat_df.index:
                                gene_kcat_list.append(gene_max_kcat_df.loc[eachdata,'kcat'])
                                gene_list.append(eachdata)
                            else:
                            #使用GO-level gene-GO
                                GO_list=list(gene_GO[gene_GO['gene']==eachdata]['GO'])
                                same_level_GO_df = pd.DataFrame()
                                for eachgo in GO_list:
                                    if eachgo in GO_level_space_out.index:
                                        go_level=GO_level_space_out.loc[eachgo,'level']
                                        go_type=GO_level_space_out.loc[eachgo,'namespace']
                                        tmp1=GO_level_space_out[GO_level_space_out['namespace']==go_type]
                                        tmp2=tmp1[tmp1['level']==go_level]
                                        kcat_list=[]
                                        for eachgo2 in tmp2.index:
                                            if eachgo2 in list(GO_ec_kcat['GO']):
                                                kcat_list.append(np.max(GO_ec_kcat[GO_ec_kcat['GO']==eachgo2]['kcat']))
                                        if len(kcat_list)>1:
                                            same_level_GO_df.loc[eachgo,'kcat']=np.max(kcat_list)
    
                                if same_level_GO_df.shape[0] >0:
                                    gene_kcat_list.append(np.max(same_level_GO_df['kcat']))
                                    gene_list.append(eachdata)

                if len(gene_kcat_list)>1:
                    reaction_max_df.loc[reaction.id,'kcat'] = np.max(gene_kcat_list)
                    reaction_max_df.loc[reaction.id,'ec'] = 'no ec'
                    reaction_max_df.loc[reaction.id,'SOURCE'] = 'gene-GO or gene-kcat'      
                else:
                    #使用所有酶的中值
                    reaction_max_df.loc[reaction.id,'kcat'] = np.median(ec_max_kcat_df['kcat'])
                    reaction_max_df.loc[reaction.id,'ec'] = 'no ec'
                    reaction_max_df.loc[reaction.id,'SOURCE'] = 'median'
    else: 
        reaction_ids = reaction.annotation["ec-code"]
        # If only one EC number is given, set the EC number string to
        # a list in order to make it work with the following code lines
        if type(reaction_ids) is str:
            reaction_ids = [reaction_ids]
        # Get all EC numbers which do not contain a - wildcard, such as
        # in 2.1.1.-
        # These wildcarded EC numbers are in general too permissive in order
        # to get useful kcats
        eligible_reaction_ids = [x for x in reaction_ids if "-" not in x]
        if len(eligible_reaction_ids) == 0:
            eligible_reaction_ids = [x for x in reaction_ids]

        # Go through each reaction ID :D
        for reaction_id in eligible_reaction_ids:
            # If the EC number could not be found in the given EC number<->kcat
            # database, print it and proceed with the next eligible EC number
            if reaction_id not in ec_max_kcat_df.index:
                if reaction_id in list(GO_ec_kcat['ec']):
                    #使用GO-level ec-GO
                    print(reaction_id)
                    GO_list = list(GO_ec_kcat[GO_ec_kcat['ec']==reaction_id]['GO'])
                    same_level_GO_df = pd.DataFrame()
                    for eachgo in GO_list:
                        go_level=GO_level_space_out.loc[eachgo,'level']
                        go_type=GO_level_space_out.loc[eachgo,'namespace']
                        tmp1=GO_level_space_out[GO_level_space_out['namespace']==go_type]
                        tmp2=tmp1[tmp1['level']==go_level]
                        kcat_list=[]
                        for eachgo2 in tmp2.index:
                            if eachgo2 in list(GO_ec_kcat['GO']):
                                kcat_list.append(np.max(GO_ec_kcat[GO_ec_kcat['GO']==eachgo2]['kcat']))
                        if len(kcat_list)>1:
                            same_level_GO_df.loc[eachgo,'kcat']=np.max(kcat_list)
                    reaction_max_df.loc[reaction.id,'kcat'] = np.max(same_level_GO_df['kcat'])
                    reaction_max_df.loc[reaction.id,'ec'] = reaction_id
                    reaction_max_df.loc[reaction.id,'SOURCE'] = 'ec-GO'

                else:
                    # reaction gpr 
                    gpr_list = reaction.gene_reaction_rule.split(' ')
   
                    gene_list=[]
                    gene_kcat_list=[]
                    for eachdata in gpr_list:
                        if len(eachdata)>4:
                            if eachdata not in gene_list:
                                #使用 gene-kcat
                                if eachdata in gene_max_kcat_df.index:
                                    gene_kcat_list.append(gene_max_kcat_df.loc[eachdata,'kcat'])
                                    gene_list.append(eachdata)
                                else:
                                #使用GO-level gene-GO
                                    GO_list=list(gene_GO[gene_GO['gene']==eachdata]['GO'])
                                    same_level_GO_df = pd.DataFrame()
                                    for eachgo in GO_list:
                                        if eachgo in GO_level_space_out.index:
                                            go_level=GO_level_space_out.loc[eachgo,'level']
                                            go_type=GO_level_space_out.loc[eachgo,'namespace']
                                            tmp1=GO_level_space_out[GO_level_space_out['namespace']==go_type]
                                            tmp2=tmp1[tmp1['level']==go_level]
                                            kcat_list=[]
                                            for eachgo2 in tmp2.index:
                                                if eachgo2 in list(GO_ec_kcat['GO']):
                                                    kcat_list.append(np.max(GO_ec_kcat[GO_ec_kcat['GO']==eachgo2]['kcat']))
                                            if len(kcat_list)>1:
                                                same_level_GO_df.loc[eachgo,'kcat']=np.max(kcat_list)
   
                                    if same_level_GO_df.shape[0] >0:
                                        gene_kcat_list.append(np.max(same_level_GO_df['kcat']))
                                        gene_list.append(eachdata)

                    if len(gene_kcat_list)>1:
                        reaction_max_df.loc[reaction.id,'kcat'] = np.max(gene_kcat_list)
                        reaction_max_df.loc[reaction.id,'ec'] = reaction_id
                        reaction_max_df.loc[reaction.id,'SOURCE'] = 'gene-GO or gene-kcat'      
                    else:
                        #使用所有酶的中值
                        reaction_max_df.loc[reaction.id,'kcat'] = np.median(ec_max_kcat_df['kcat'])
                        reaction_max_df.loc[reaction.id,'ec'] = reaction_id
                        reaction_max_df.loc[reaction.id,'SOURCE'] = 'median'
            else:
                # Otherwise, get the reaction ID entry from the given database :D
                reaction_max_df.loc[reaction.id,'kcat'] = ec_max_kcat_df.loc[reaction_id,'kcat']
                reaction_max_df.loc[reaction.id,'ec'] = reaction_id
                reaction_max_df.loc[reaction.id,'SOURCE'] = ec_max_kcat_df.loc[reaction_id,'SOURCE']
reaction_max_df.to_csv('./analysis/reaction_max_df.csv')